In [1]:
import os

import pandas as pd
import numpy as np
import openpyxl as ox
import re

from openpyxl import load_workbook
from openpyxl import workbook
from openpyxl.styles import Font
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.cell.cell import WriteOnlyCell

from datetime import datetime, timedelta

import warnings
warnings.filterwarnings("ignore")

import string

import pyodbc


In [2]:
conn = pyodbc.connect(r'Driver={SQL Server};Server=MSK1-BIDB01;Database=DWH;Trusted_Connection=yes;')
cursor = conn.cursor()

Employees = f'''

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED

SELECT [EMPLOYEES]
      ,[LOG]
      ,[SP]

      ,CASE

       WHEN KC like '%4%' THEN 'КЦ 4'
       WHEN KC like '%3%' THEN 'КЦ 3'
       WHEN KC like '%центр 3%' THEN 'КЦ 3'
         WHEN KC like '%продаж 18%' THEN 'ОП 18'
             
       ELSE KC
  END
  as "КЦ"

         ,CASE

       WHEN [KC] like '%4%' THEN SUBSTRING([ОП],CHARINDEX(' ',[ОП])+1,LEN([ОП])) + ' ВР'
       WHEN ([ОП] in ('ОП 9')) and ([GP] in ('.1')) THEN [ОП] + [GP]
       WHEN ([ОП] in ('ОП 2')) and ([GP] in ('.1')) THEN 'ОП 8'
       WHEN ([ОП] in ('ОП 10')) and ([GP] in ('.2')) THEN [ОП] + [GP]
       WHEN ([ОП] in ('ОП 5')) and ([GP] in ('.1')) THEN [ОП] + [GP]
       WHEN ([ОП] in ('ОП 14')) and ([GP] in ('.1')) THEN [ОП] + [GP]
       WHEN ([ОП] in ('3 ЯР')) and ([GP] in ('.1')) THEN '3.1 ЯР'
         WHEN ([KC] in ('Отдел прямых продаж'))  THEN 'ОПП'
           WHEN [KC] like '%продаж 18%'  THEN 'ОП 18'  
       ELSE [ОП]
  END
  as "ОП"
      
               ,[PHONE]
      ,[STATUS]
      ,[ID_EMPL]
      ,[ID_ORG]

  FROM [DWH].[dbo].[KHTS_EMPL]
  where 
  --KC like '%4%' 
  [SP] in ('КД')

'''

Employees = pd.read_sql_query(Employees, conn)
Employees = Employees.rename(columns={'EMPLOYEES': 'Ответственный'})
Employees = Employees[['Ответственный', 'КЦ', 'ОП']]


over = pd.read_excel('C:\\Users\\ADavydovskiy\\scripts\\Выгрузки\\Overdue_affairs_in_deals.xlsx', sheet_name='Лист3')

In [3]:
over

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,NaN,NaN,1 февраля 2023 г. 08:41:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Сделки за период с 1 января 2023 г. по 30 янв...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Стажер 4 КД,3,0,3,0.0,202000.0
6848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5389236,NaN,Да,0.0,0.0
6849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5389241,NaN,Да,0.0,32000.0
6850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5411850,NaN,Да,0.0,170000.0


In [4]:
over = over[5:].reset_index()
over

,index,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,5,NaN,NaN,Департамент,NaN,NaN,Подразделение,Отдел/Рабочая Группа,Менеджер,Всего сделок,Сделок без дел,Сделок с просроченными делами,Сумма товара сделок без дел,Сумма товара сделок с просроченными делами
1,6,NaN,NaN,Коммерческий департамент,NaN,NaN,Итого по департаменту :,NaN,NaN,6611,66,365,1388170.0,16317238.0
2,7,NaN,NaN,NaN,NaN,NaN,,Итого по подразделению :,NaN,1,0,0,0.0,0.0
3,8,NaN,NaN,NaN,NaN,NaN,NaN,,Итого по Отделу/раб. группе :,1,0,0,0.0,0.0
4,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Трушкин Александр Сергеевич,1,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6842,6847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Стажер 4 КД,3,0,3,0.0,202000.0
6843,6848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5389236,NaN,Да,0.0,0.0
6844,6849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5389241,NaN,Да,0.0,32000.0
6845,6850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5411850,NaN,Да,0.0,170000.0


In [5]:
new_header = over.iloc[0]

In [6]:
over = over[1:]
over

,index,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
1,6,NaN,NaN,Коммерческий департамент,NaN,NaN,Итого по департаменту :,NaN,NaN,6611,66,365,1388170.0,16317238.0
2,7,NaN,NaN,NaN,NaN,NaN,,Итого по подразделению :,NaN,1,0,0,0.0,0.0
3,8,NaN,NaN,NaN,NaN,NaN,NaN,,Итого по Отделу/раб. группе :,1,0,0,0.0,0.0
4,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Трушкин Александр Сергеевич,1,0,0,0.0,0.0
5,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5384500,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6842,6847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Стажер 4 КД,3,0,3,0.0,202000.0
6843,6848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5389236,NaN,Да,0.0,0.0
6844,6849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5389241,NaN,Да,0.0,32000.0
6845,6850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5411850,NaN,Да,0.0,170000.0


In [7]:
over.columns = new_header
over

,5,NaN,NaN,Департамент,NaN,NaN,Подразделение,Отдел/Рабочая Группа,Менеджер,Всего сделок,Сделок без дел,Сделок с просроченными делами,Сумма товара сделок без дел,Сумма товара сделок с просроченными делами
1,6,NaN,NaN,Коммерческий департамент,NaN,NaN,Итого по департаменту :,NaN,NaN,6611,66,365,1388170.0,16317238.0
2,7,NaN,NaN,NaN,NaN,NaN,,Итого по подразделению :,NaN,1,0,0,0.0,0.0
3,8,NaN,NaN,NaN,NaN,NaN,NaN,,Итого по Отделу/раб. группе :,1,0,0,0.0,0.0
4,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Трушкин Александр Сергеевич,1,0,0,0.0,0.0
5,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5384500,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6842,6847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Стажер 4 КД,3,0,3,0.0,202000.0
6843,6848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5389236,NaN,Да,0.0,0.0
6844,6849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5389241,NaN,Да,0.0,32000.0
6845,6850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5411850,NaN,Да,0.0,170000.0


In [8]:
over = over[['Менеджер', 'Всего сделок', 'Сделок без дел', 'Сделок с просроченными делами']]
over.columns = ['Ответственный', 'Всего сделок', 'Сделок без дел', 'Сделок с просроченными делами']

In [ ]:
over = over.dropna()
over

In [ ]:
over = over.loc[over['Ответственный'].str.contains('Итого')==False]
over

In [ ]:
new_df = over['Ответственный'].str.split(' ',expand=True)
over

In [ ]:
new_df.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8']
over

In [ ]:
over['Ответственный'] = new_df['5'].map(str) + ' ' + new_df['6'].map(str) + ' ' + new_df['7'].map(str)
over

In [ ]:
over['Ответственный'] = over['Ответственный'].replace("     ", "")
over = over.merge(Employees, how='left', on='Ответственный')
over

In [ ]:
over = over[['КЦ', 'ОП', 'Ответственный', 'Всего сделок', 
             'Сделок без дел', 'Сделок с просроченными делами']]
over

In [4]:
over['Всего сделок'].count()

202

In [2]:
over_pivot = over.pivot_table(index='ОП',
                              values=['Сделок с просроченными делами', 
                                       'Сделок без дел', 'Всего сделок'],
                              aggfunc='count')

In [3]:
over_pivot

,Всего сделок,Сделок без дел,Сделок с просроченными делами
ОП,,,
1 ВР,7,7,7
1 ЯР,11,11,11
2 ЯР,10,10,10
3 ВР,10,10,10
4 ЯР,12,12,12
5 ЯР,2,2,2
Группа качества,2,2,2
Группа по сохранению клиентов,4,4,4
ОП 10,9,9,9
